# Testing tig and forge on a collection
## PO.DAAC Data Publication Team
#### *Author: Dean Henze*

This notebook can be used to test
1. Generation of a tig-forge configuration file from a test granule.
2. Use of the config file and a test granule with tig to generate image thumbnails (used in HiTIDE GUI).
3. Use of the config file and a test granule with forge to generate granule footprints (used to visualize individual granule footprints in both HiTIDE and Earthdata Search).

Tests 2 and 3 can be run separately of each other. For example, a DPub engineer might decide that their collection requires footprints for Earthdata search, but won't be part of HiTIDE (and therefore doesn't require image thumbnails).

This notebook uses the tig and forge software, which are maintained by the TVA team. The tig and forge repos can be found at:
* https://github.com/podaac/tig
* https://github.com/podaac/forge

tig is available as a python package and instructions on obtaining it are in the following section. forge is a java program and is installed as part of Section 3. 

# Import Python Libraries

### Packages other than `podaac-tig`

In [ ]:
import os
import shutil
import json
import numpy as np
import matplotlib.pyplot as plt
import cartopy
%matplotlib inline

### `podaac-tig` package

The podaac-tig library is in https://pypi.org/project/podaac-tig/ , so pypi should be added to your `~/pip/pip.conf` file before using pip to install the package. E.g. the `pip.conf` should have the lines
```
[global]
index-url = https://pypi.org/simple
trusted-host = pypi.org
```
Then the podaac-tig library can be installed with 
```
pip install podaac-tig==0.8.0
```

In [ ]:
from podaac.tig import generate_hitide_config
from podaac.tig import tig

# 1. Generate config file

The config file can be used to either generate footprints with forge, or thumbnails with tig (which ever are relevant to your data set). The config file will be saved as `<collection-short-name>.cfg`. 

Required files/directories to have in the same directory as this notebook:
1. One or more test granules from the collection, in a folder `granules/`. This notebook assumes the file extensions are one of `.nc`, `.hdf`, or `.h5`.
2. A CSV file named `vars.csv` with variable names to add to HiTIDE, along with the min/max values to use for each of their colorscales. The column headers should be `variable`, `min`, `max`. If no variables will go into HiTIDE, keep the CSV file empty with only a header row.

Then, modify the lines in the following block of code (no other code in this Section needs modification).

In [ ]:
## User to specify some input args to the generate config function:
    # collection shortname:
dataset_id = "ASCATB_ESDR_L2_WIND_STRESS_V1.1" 
    # csv file for min/max
include_image_variables = "./vars.csv" 
    # str's, names of coords in file:
longitude = 'lon' 
latitude = 'lat'
time = 'time'
    # footprint strategy. str, usually 'periodic'. For a full list of strategy options, see section 5.1.
footprint_strategy = 'periodic'

No need for modification from here on:

In [ ]:
# Clone the forge-tig-configuration repo (https://github.com/podaac/forge-tig-configuration):
!git clone git@github.com:podaac/forge-tig-configuration.git

In [ ]:
# Path to directory of granules to test:
dir_granules_test = "./granules/"
# Path to color palettes directory (requires that the forge-tig-configuration repo was cloned into this directory):
palette_dir = "./forge-tig-configuration/palettes"
# Path to the config file
config_file="./" + dataset_id + ".cfg"

In [ ]:
# Generaterate the config file and print file contents to notebook:
gran_paths = [dir_granules_test + f for f in os.listdir(dir_granules_test) if f.endswith((".nc", ".hdf", ".h5"))]

cfg = generate_hitide_config.generate_hitide_config(
    gran_paths[0],
    dataset_id,
    include_image_variables,
    longitude, latitude, time,
    footprint_strategy
    )

with open(dataset_id + '.cfg', 'w') as fp:
    json.dump(cfg, fp, indent=4)

# 2. Tig thumbnail image generation
No code in this section should need modification other than special cases.

In [ ]:
# save the thumbnail images here:
output_dir="./images/" 
!mkdir $output_dir

In [ ]:
## Generate thumbnails for all test granules:
for g in gran_paths:
    # Specify some additional inputs for the tig class object:
    input_file = g # test granule
    granule_id = input_file.split('/')[-1]

    # Run tig and generate the images:
    image_gen = tig.TIG(input_file, output_dir, config_file, palette_dir)
    image_gen.generate_images(granule_id=granule_id)

In [ ]:
## Test plots of all images:
image_files = [output_dir+f for f in os.listdir(output_dir) if f.endswith(".png")]
fig, axes = plt.subplots(len(image_files),1, figsize=(14, 6*len(image_files)))
for f, ax in zip(image_files, axes):
    img = plt.imread(f)
    im = ax.imshow(img)
    ax.set_title(f.split(".")[-2])
    #plt.axis("off")
plt.show()

# 3. Forge footprint generation
No code in this section should need modification other than special cases.

## 3.1 Download and run forge software:
Note, `forge` is Java software and requires Java 11+

In [ ]:
!curl -L -O https://github.com/podaac/forge/releases/download/0.9.0/footprint_0.9.0.jar

In [ ]:
# Directory to store output in:
!mkdir footprints

In [ ]:
# Run forge for all test granules:
for g in gran_paths:
    
    granule_id = g.split('/')[-1]
    
    # Specify output filenames:
    fn_forge_output = "footprints/" + granule_id + ".forge_output.txt" # All output from forge
    fn_wkt = "footprints/" + granule_id + ".footprint.wkt" # only the footprint polygon results

    # Run forge and save output to file:
    !java -cp ./footprint_0.9.0.jar FootprintCLI $g $config_file >> $fn_forge_output
    # Extract footprint (WKT polygon) from forge output
    !cat $fn_forge_output | grep POLYGON >> $fn_wkt

## 3.2 Unpack WKT polygon info and plot

Unpacking assumes that the WKT file has the following format:
```
"Polygon ((<lon1 lat1>, <lon2 lat2>, ... <lonN latN>))"
```
where e.g. `<lat1 lon1>` is the first latitue longitude coordinate separated by a blank `" "`, and each latitude longintude pair is separated by a comma.

In [ ]:
def unpack_poly(fn_wkt):
    """
    Unpack polygon lat, lon point info for a WKT footprint file into two lists. 
    'fn_wkt' is the path (str) to the WKT file.
    """
    
    with open(fn_wkt, 'r') as f:
        footprint_wkt = f.read().split("\n")
        
    if footprint_wkt[0][:7] == 'POLYGON': # If file starts off as expected, unpack.
        
        lons, lats = [], []
        temp_fp = footprint_wkt[0].split("((")[1].split("))")[0]
        
        for polypoint in temp_fp.split(","):
            lon_p, lat_p = polypoint.split(" ")[-2:]
            lons.append(float(lon_p))
            lats.append(float(lat_p))

        # Add last point to end so can plot a closed polygon
        lons.append(lons[0])
        lats.append(lats[0])

        return lons, lats
    
    elif "LINESTRING" in footprint_wkt[0]:
        
        temp_fp = footprint_wkt[0].split("(")[1].split(")")[0]
        
        for polypoint in temp_fp.split(","):
            lon_p, lat_p = polypoint.split(" ")[-2:]
            lons.append(float(lon_p))
            lats.append(float(lat_p))
            
        return lons, lats
        
    else:
        print("WKT file format is not as expected. Please inspect file contents.")
        return None, None

In [ ]:
wkt_files = ["./footprints/"+f for f in os.listdir("./footprints/") if f.endswith(".wkt")]

for fn_wkt in wkt_files:
    # Unpack polygon lat, lon point info into two lists:
    lons, lats = unpack_poly(fn_wkt) 

    # Plot polygon:
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=cartopy.crs.PlateCarree(central_longitude=0))
    ax.plot(lons, lats)
    
    ax.gridlines()
    ax.add_feature(cartopy.feature.OCEAN, zorder=0, facecolor='gainsboro')
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', linewidth=0.5, facecolor='gainsboro')

# 4. Upload config file to forge-tig-configuration repo
Create a PR for tig-forge-configuration repo: https://github.com/podaac/forge-tig-configuration

1. Copy each short name .cfg file to the forge-tig-configuration repo under "config-files"
2. Update the CHANGELOG.md in forge-tig-configuration listing your changes
3. Commit and push the new updates in forge-tig-configuration to a new branch named "feature/\<name\>"
    NOTE: The \<name\> can be anything you want to describe the collection you are adding
4. Open a PR in forge-tig-configuration for this new branch
5. In the PR, include reviewers James Wood and Simon Liu, and let them know it's ready in Slack

In [ ]:
# 1. Copies the .cfg file to the required folder:
shutil.copyfile(config_file, "./forge-tig-configuration/config-files/" + config_file.split("/")[-1])

Update `CHANGELOG.md`:
Add the following lines, modified as needed to reflect changes, e.g.:
```
## [Released] - yyyy-mm-dd

### Added
 - Added config for <shortname>
### Changed
### Deprecated
### Removed
### Fixed
### Security

```

Commands for committing and pushing to new branch once inside the directory `forge-tig-configuration`:

```
cd forge-tig-configuration
git branch feature/<name>
git checkout feature/<name>
git add config-files/<shortname.cfg> CHANGELOG.md
git commit -m "<commit message>"
git push -u origin feature/<name>
```

Then create a PR in https://github.com/podaac/forge-tig-configuration, include reviewers James Wood and Simon Liu, and let them know it's ready in Slack

# 5. Other notes

## 5.1 Footprint Strategies

The current options for footprint strategy are *periodic*, *linestring*, *polar*, *polarsides*, *smap*, and *swot_linestring*.

Below are examples of collections that use each strategy:

**periodic**

* AQUARIUS_L2_SSS_V4
* AQUARIUS_L2_SSS_V5
* QSCAT_LEVEL_2B_OWV_COMP_12_KUSST_LCRES_4.1
* QSCAT_LEVEL_2B_OWV_COMP_12_LCR_3.1
* VIIRS_NPP-NAVO-L2P-v1.0
* AVHRRF_MC-STAR-L2P-v2.80
* ASCATC-L2-25km
* AMSR2-REMSS-L2P-v8.2
* VIIRS_NPP-NAVO-L2P-v3.0
* ASCATA-L2-Coastal
* AMSR2-REMSS-L2P_RT-v8.2
* SWOT_SIMULATED_L2_KARIN_SSH_GLORYS_CALVAL_V1
* VIIRS_NPP-OSPO-L2P-v2.41
* ASCATC-L2-Coastal
* MODIS_T-JPL-L2P-v2019.0
* VIIRS_N20-OSPO-L2P-v2.61
* RSCAT_LEVEL_2B_OWV_COMP_12_V1.2
* AVHRR_SST_METOP_B-OSISAF-L2P-v1.0
* VIIRS_NPP-JPL-L2P-v2016.2
* VIIRS_NPP-NAVO-L2P-v2.0
* RSCAT_LEVEL_2B_OWV_COMP_12_V1.1
* ASCATB-L2-Coastal
* MODIS_A-JPL-L2P-v2019.0


**linestring**

* JASON-1_L2_OST_GPS_E.
* JASON_CS_S6A_L2_ALT_HR_STD_OST_NRT_F
* JASON-1_L2_OST_GPN_E
* JASON_CS_S6A_L2_ALT_LR_STD_OST_STC_F
* JASON_3_L2_OST_OGDR_GPS
* JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F
* JASON-1_L2_OST_GPR_E
* SWOT_SIMULATED_L2_NADIR_SSH_GLORYS_CALVAL_V1
* JASON_CS_S6A_L2_ALT_HR_RED_OST_STC_F
* OSTM_L2_OST_OGDR_GPS
* SWOT_SIMULATED_L2_NADIR_SSH_GLORYS_SCIENCE_V1
* JASON_CS_S6A_L2_ALT_HR_STD_OST_STC_F
* JASON_CS_S6A_L2_AMR_RAD_STC
* MERGED_TP_J1_OSTM_OST_CYCLES_V42
* SWOT_SIMULATED_L2_NADIR_SSH_ECCO_LLC4320_SCIENCE_V1
* JASON_CS_S6A_L2_ALT_LR_STD_OST_NRT_F
* ALTIKA_SARAL_L2_OST_XOGDR
* JASON_CS_S6A_L2_ALT_LR_RED_OST_STC_F
* SWOT_SIMULATED_L2_NADIR_SSH_ECCO_LLC4320_CALVAL_V1
* JASON_CS_S6A_L2_AMR_RAD_NRT
* JASON_CS_S6A_L2_ALT_HR_RED_OST_NRT_F

**polar**

* NAVO-L2P-AVHRR19_G
* AVHRR19_G-NAVO-L2P-v1.0

**polarsides**

* OS2_OSCAT_LEVEL_2B_OWV_COMP_12_V2
* AVHRRMTA_G-NAVO-L2P-v1.0
* AVHRRMTB_G-NAVO-L2P-v1.0
* SMAP_JPL_L2B_NRT_SSS_CAP_V5

**smap**

* SMAP_RSS_L2_SSS_V5                
* SMAP_RSS_L2_SSS_V4            
* SMAP_JPL_L2B_SSS_CAP_V5

**swot_linestring**

* SWOT_L2_NALT_IGDR_1.0
* SWOT_L2_NALT_IGDR_2.0
* SWOT_L2_NALT_OGDR_2.0
* SWOT_L2_NALT_GDR_SSHA_1.0
* SWOT_L2_NALT_IGDR_SSHA_1.0
* SWOT_L2_NALT_OGDR_SSHA_2.0
* SWOT_L2_NALT_OGDR_1.0
* SWOT_L2_NALT_IGDR_SSHA_2.0
* SWOT_L2_NALT_GDR_SSHA_2.0
* SWOT_L2_NALT_OGDR_SSHA_1.0